In [ ]:
import numpy as np

# Bayes' rule for discrete hypotheses

Posterior = Likelihood * Prior / Marginal probability of the data

$$ P(h|d)=\frac{P(d|h)\cdot P(h)}{P(d)}$$

## Example: Medical diagnosis

We have a patient who coughed. What is the probability that she has lung cancer?

### Data ($d$):
The patient coughed.

### Hypotheses ($h$):
1. Healthy ($h_1$)
1. Cold ($h_2$)
1. Lung cancer ($h_3$)

### Prior probabilities ($P(h)$):
*From prior experience, we know that out of 100 random patients, 90 are healthy, 9 have a cold, and 1 has lung cancer.*
1. $P(h_1)=0.90$
1. $P(h_2)=0.09$
1. $P(h_3)=0.01$

### Likelihood function ($P(d|h)$):
*If a patient is healthy, it is very unlikely that she coughs (1 in 100), if she has a cold it is probable (50 in 100), and if she has lung cancer it is very likely (99 in 100) that she coughs.*
1. $P(d|h_1)=0.01$
1. $P(d|h_2)=0.5$
1. $P(d|h_3)=0.99$

### Posterior ($P(h|d)$)

Inserting the values to Bayes' rule from above gives:

The probability that the patient is **healthy** is 
$P(h_1|d)=\frac{.01 * .9}{P(d)}=\frac{.009}{P(d)}$

The probability that the patient has a **cold** is 
$P(h_2|d)=\frac{.5 * .09}{P(d)}=\frac{.045}{P(d)}$

The probability that the patient has **lung cancer** is 
$P(h_3|d)=\frac{.99 * .01}{P(d)}=\frac{.0099}{P(d)}$


Because the **marginal probability of the data** $P(d)$ is the same in each case, we only need to compute it once: $$P(d)=\sum_h P(d|h) \cdot P(h) = .009 + .045 + .0099 = .0639$$ 
This constant guarantees that the posterior probabilities sum to 1, so that they form a proper probability distribution. Now we have the posterior probabilities:

The probability that the patient is **healthy** is 
$P(h_1|d)=\frac{.009}{.0639} \approx .14$

The probability that the patient has a **cold** is 
$P(h_2|d)=\frac{.045}{.0639} \approx .70$

The probability that the patient has **lung cancer** is 
$P(h_3|d)=\frac{.0099}{.0639} \approx .15$

In [ ]:
# number of hypotheses
n = 3

# prior P(h)
prior = [0.9, 0.09, 0.01]

# likelihood P(d|h)
p_d_given_h = [0.01, 0.5, 0.99]

# in-between step
proto = [p_d_given_h[i] * prior[i] for i in range(n)]

# marginal probability of the data
p_d = sum(proto)

# posterior
posterior = [proto[i] / p_d for i in range(n)]

print("P(Healthy|Cough)={}".format(posterior[0]))
print("P(Cold|Cough)={}".format(posterior[1]))
print("P(LungCancer|Cough)={}".format(posterior[2]))

In [ ]:
# What if the patient hadn't coughed?

# update likelihood
# P(no cough|h_1) = 1 - P(cough|h_1)
# P(no cough|h_2) = 1 - P(cough|h_2)
# P(no cough|h_3) = 1 - P(cough|h_3)
p_d_given_h = [1 - 0.01, 1 - 0.5, 1 - 0.99]

proto = [p_d_given_h[i] * prior[i] for i in range(n)]
p_d = sum(proto)
posterior = [proto[i] / p_d for i in range(n)]

print("P(Healthy|No cough)={}".format(posterior[0]))
print("P(Cold|No cough)={}".format(posterior[1]))
print("P(LungCancer|No cough)={}".format(posterior[2]))

## General functions
It can be that simple!

In [ ]:
def normalize(x):
    return x / sum(x)

def posterior(prior, likelihood):
    proto = likelihood * prior
    return normalize(proto)

In [ ]:
## The functions only work with numpy arrays
prior = np.array([0.9, 0.09, 0.01])
likelihood = np.array([0.01, 0.5, 0.99])
posterior(prior, likelihood)

## Example: Weather
In the 'Joint, Conditional, Marginal Distributions' notebook, we had the probability of traffic given certain weather P(T|W) and the probability of certain weather in general P(W). My office does not have any windows so I can't see the weather. But I know there is *no traffic* because my friend told me in a text message. With Bayes' rule I can compute what weather it is, given that there no traffic. My life is great.

In [ ]:
## We are looking for P(W|T)

## Copying info from other notebook
def P_W(weather):
    if weather == "sunny":
        return 0.7
    if weather == "cloudy":
        return 0.2
    if weather == "stormy":
        return 0.1
def P_T_given_W(weather, traffic):
    states = {
        ("sunny", "yes") : 0.143,
        ("cloudy", "yes"): 0.5,
        ("stormy", "yes"): 1.0,
        ("sunny", "no")  : 0.857,
        ("cloudy", "no") : 0.5,
        ("stormy", "no") : 0.0,
    }
    return states[(weather, traffic)]

In [ ]:
## hypotheses
hypotheses = ("sunny", "cloudy", "stormy")

## data
traffic = 'no' 

## turn into numpy arrays
prior = np.array([P_W(weather) for weather in hypotheses])
likelihood = np.array([P_T_given_W(weather, traffic) for weather in hypotheses])

## compute posterior belief
belief = posterior(prior, likelihood)

In [ ]:
for i, v in enumerate(hypotheses):
    print(v + ': ' + str(belief[i]))

It is probably sunny outside! :)

## Bayes rule in log space

When computing the $P(d|h)\cdot P(h)$, the values we are multiplying are very small (e.g. because we have many hypotheses or very unlikely data). Then the result becomes even smaller and the computer might not be able to represent such small number.

The solution is to represent the probabilities in log space (and there is even some evidence that the brain is representing probabilities in log space too). A really small probability is easily and accurately represented by a negative number in log space:

In [ ]:
x = 0.0000000000000156
np.log(x)

This means for Bayes' rule:

$$ P(h|d)=\frac{P(d|h)\cdot P(h)}{P(d)}$$

turns into 

$$ \log P(h|d)=\log P(d|h) + \log P(h) - \log P(d)$$

So far so good, but to compute $P(d)$ we took the sum of all the $P(d|h)\cdot P(h)$ values for each $h$. Since multiplications turn into additions in log space, just taking the sum of the $\log P(d|h) + \log P(h)$ values naively, you may encounter underflows or overflows. Even if you work in log-space, the limited precision of computers may not be enough and the result will be INF or -INF. Instead we use the `logsumexp` function from the `scipy` package in our `normalize` function. Our functions from above then turn into:

In [ ]:
from scipy.special import logsumexp

def log_normalize(x):
    return x - logsumexp(x)
#     return x - np.log(np.sum(np.exp(x))) # <-- this wouldn't work

def log_posterior(log_prior, log_likelihood):
    proto = log_likelihood + log_prior
    return log_normalize(proto)

After the log_normalization it is safe to turn the log probabilities back into probabilities via $P(x) = exp(log P(x))$.

In [ ]:
## try out
log_prior = np.log(np.array([0.5, 0.25, 0.25]))
log_likelihood = np.array([-2000, -2000, -3000])
np.exp(log_posterior(log_prior, log_likelihood))

You won't have to worry about the details of the log-sum-exp trick for this class, but if you're curious, see [this blog post](https://gregorygundersen.com/blog/2020/02/09/log-sum-exp/) for further details